In [1]:
import cv2 as cv
import numpy as np
import functools
import matplotlib.pyplot as plt
import random

In [2]:
img = cv.imread("./unsplash.jpg", 0)
h, w = img.shape

In [3]:
def rescale_to_fit(image):
    target_w, target_h = 1920, 1080
    h, w = image.shape
    # If the image is larger than the target display size, rescale it
    if w > target_w or h > target_h:
    # Calculate the scaling factor for width and height
        scale_w = target_w / w
        scale_h = target_h / h
        scale = min(scale_w, scale_h)

        # Resize the image using the scaling factor
        return cv.resize(image, None, fx=scale, fy=scale)

In [4]:
#Original output
cv.imshow("Unsplash Original", rescale_to_fit(img))
cv.waitKey(0)
cv.destroyAllWindows()

In [5]:
def cropping(image):
    h, w = image.shape
    h = h//8*8
    w = w//8*8
    return image[0:h, 0:w]

img = cropping(img)

# #Cropped output
# cv.imshow("Unsplash 8 Divisible", img)
# cv.waitKey(0)
# cv.destroyAllWindows()

In [6]:
h, w= img.shape
sub_height, sub_width = (h//4, w//2)

In [7]:
def divide_and_shuffle(input_image, sub_height, sub_width, shuffle=False):
    parts_list = list()
    for x in range(4):
        parts_list.append(input_image[(x)*sub_height:(x+1)*sub_height, 0:sub_width])
        parts_list.append(input_image[(x)*sub_height:(x+1)*sub_height, sub_width:])
    
    return (random.sample(parts_list, len(parts_list)) if shuffle == True else parts_list)


parts_list = divide_and_shuffle(img, sub_height, sub_width, True)

In [8]:
def combine_shuffled_image(parts_list):
    for x in range(0, 8, 2):
        img_new = cv.hconcat([parts_list[x], parts_list[x+1]])
        parts_list.append(img_new)
    
    for x in range(8):
        parts_list.pop(0)
    
    for x in range(0, 4, 2):
        img_new = cv.vconcat([parts_list[x], parts_list[x+1]])
        parts_list.append(img_new)
        
    for x in range(4):
        parts_list.pop(0)
        
    return_img = cv.vconcat([parts_list[0], parts_list[1]])
    parts_list.clear()
    return return_img

img_new = combine_shuffled_image(parts_list)

In [9]:
#Shuffled output
cv.imshow("Unsplash Shuffled", rescale_to_fit(img_new))
cv.waitKey(0)
cv.destroyAllWindows()

In [10]:
@functools.total_ordering
class Part:
    top_neighbour = [None] * 2
    bottom_neighbour = [None] * 2
    left_neighbour = [None] * 2
    right_neighbour = [None] * 2
    data = None
    top = None
    bottom = None
    left = None
    right = None

    def set_neighbour(self, side, neighbour, percentage):
        if side == "T":
            self.top_neighbour = [neighbour, percentage]
        if side == "B":
            self.bottom_neighbour = [neighbour, percentage]
        if side == "L":
            self.left_neighbour = [neighbour, percentage]
        if side == "R":
            self.right_neighbour = [neighbour, percentage]

    def get_neighbour(self, side):
        if side == "T":
            return self.top_neighbour[0]
        if side == "B":
            return self.bottom_neighbour[0]
        if side == "L":
            return self.left_neighbour[0]
        if side == "R":
            return self.right_neighbour[0]
        
    def get_percentage(self, side):
        if side == "T":
            return self.top_neighbour[1]
        if side == "B":
            return self.bottom_neighbour[1]
        if side == "L":
            return self.left_neighbour[1]
        if side == "R":
            return self.right_neighbour[1]

    def __init__(self, top, bottom, left, right, data):
        self.top = top
        self.bottom = bottom
        self.left = left
        self.right = right
        self.data = data
        
    def __str__(self):
        return f'Top: {self.top},\nBottom: {self.bottom},\nLeft: {self.left},\nRight: {self.right}\n'
    
    def _is_valid_operand(self, other):
        return (isinstance(other, Part))

    def __eq__(self, other):
        if not self._is_valid_operand(other):
            return NotImplemented
        return np.array_equal(self.data, other.data)
    
    def __lt__(self, other):
        if not self._is_valid_operand(other):
            return NotImplemented
        return TypeError("Part does not support the operator")

In [11]:
subparts = divide_and_shuffle(img_new, sub_height, sub_width, False)
visited = list()

def part_details(subparts):
    part_info = list()
    for i in subparts:
        temp = Part(i[0:1, 0:], i[-1:, 0:], i[0:, 0:1], i[0:, -1:], i)
        part_info.append(temp)
        
    return part_info

In [12]:
def compare_edges(part_a, part_b):
    num_equal = 0
    total_elements = part_a.size
    num_equal = np.sum(part_a == part_b)
    equality_percentage = (num_equal / total_elements * 100)
    return equality_percentage

In [13]:
#Vertical neighbour check
def compare_vert(part_a, subparts):
    max_match = None
    
    #Matching bottom of part_a with top of all
    curr_percentage = 0
    max_percentage = -1
    for x in subparts:
        if x == part_a:
            continue
        else:
            curr_percentage = compare_edges(part_a.bottom, x.top)
            if curr_percentage > max_percentage:
                max_match = x
                max_percentage = curr_percentage
                part_a.set_neighbour("B", max_match, max_percentage)
    #Matching top of part_a with bottom of all
    curr_percentage = 0
    max_percentage = -1
    for x in subparts:
        if x == part_a:
            continue
        else:
            curr_percentage = compare_edges(part_a.top, x.bottom)
            if curr_percentage > max_percentage:
                max_match = x
                max_percentage = curr_percentage
                part_a.set_neighbour("T", max_match, max_percentage)

    return

In [14]:
def compare_hor(part_a, subparts):
    max_match = None
    
    #Matching right of part_a with left of all
    curr_percentage = 0
    max_percentage = -1
    for x in subparts:
        if x == part_a:
            continue
        else:
            curr_percentage = compare_edges(part_a.right, x.left)
            if curr_percentage > max_percentage:
                max_match = x
                max_percentage = curr_percentage
                part_a.set_neighbour("R", max_match, max_percentage)

    #Matching left of part_a with right of all
    curr_percentage = 0
    max_percentage = -1
    for x in subparts:
        if x == part_a:
            continue
        else:
            curr_percentage = compare_edges(part_a.left, x.right)
            if curr_percentage > max_percentage:
                max_match = x
                max_percentage = curr_percentage
                part_a.set_neighbour("L", max_match, max_percentage)

    return

In [15]:
part_info = part_details(subparts)
visited.clear()
for x in range(len(part_info)):
    if x in visited:
        continue
    else:
        visited.append(x)
        compare_hor(part_info[x], part_info)
    

In [16]:
def concat_parts(part_info, orientation):
    if orientation == "horizontal":
        while len(part_info) > 4:
            l = part_info[0].get_percentage("L")
            r = part_info[0].get_percentage("R")
            if l > r:
                l_part = part_info[0].get_neighbour("L")
                new_subpart_data = cv.hconcat([l_part.data, part_info[0].data])
                new_obj = Part(new_subpart_data[0:1, 0:], new_subpart_data[-1:, 0:], new_subpart_data[0:, 0:1], new_subpart_data[0:, -1:], new_subpart_data)
                part_info.append(new_obj)
                part_info.pop(0)
                part_info.pop(part_info.index(l_part))
            else:
                r_part = part_info[0].get_neighbour("R")
                new_subpart_data = cv.hconcat([part_info[0].data, r_part.data])
                new_obj = Part(new_subpart_data[0:1, 0:], new_subpart_data[-1:, 0:], new_subpart_data[0:, 0:1], new_subpart_data[0:, -1:], new_subpart_data)
                part_info.append(new_obj)
                part_info.pop(0)
                part_info.pop(part_info.index(r_part))

    if orientation == "vertical":
        index = 0
        perc = 100
        for y in part_info:
            if m:=y.get_percentage("T") < perc:
                perc = m
                index = part_info.index(y)
                
        length = len(part_info)
        for x in range(length - 1):
            new_subpart_data = cv.vconcat([part_info[index].data, part_info[index].get_neighbour("B").data])
            new_obj = Part(new_subpart_data[0:1, 0:], new_subpart_data[-1:, 0:], new_subpart_data[0:, 0:1], new_subpart_data[0:, -1:], new_subpart_data)
            new_obj.set_neighbour("B", part_info[index].get_neighbour("B").get_neighbour("B"), part_info[index].get_neighbour("B").get_percentage("B"))
            part_info.append(new_obj)
            index = part_info.index(new_obj)
        
        length = len(part_info)
        for x in range(length - 1):
            part_info.pop(0)

    return

In [17]:
concat_parts(part_info, "horizontal")

In [18]:
visited.clear()
for x in range(len(part_info)):
    if x in visited:
        continue
    else:
        visited.append(x)
        compare_vert(part_info[x], part_info)

In [19]:
concat_parts(part_info, "vertical")

In [20]:
# print(part_info[0].data.shape)

cv.imshow("Reconstructed Image", rescale_to_fit(part_info[0].data))
cv.waitKey(0)
cv.destroyAllWindows()